In [ ]:
import torch
import torch.nn as nn

from torch.utils.data import DataLoader, Dataset
from unsupervised_meta_learning.proto_utils import CNN_4Layer
from unsupervised_meta_learning.pl_dataloaders import UnlabelledDataModule, UnlabelledDataset

In [ ]:
 dataset_train = UnlabelledDataset('omniglot',
                                          './data/', split='train',
                                          transform=None,
                                          n_images=None,
                                          n_classes=None,
                                          n_support=1,
                                          n_query=3)

In [ ]:
dataset_train[0]['data'].shape

torch.Size([4, 1, 28, 28])

In [ ]:
dataloader_train = DataLoader(dataset_train,
                                      batch_size=5,
                                      shuffle=True,
                                      num_workers=8,
                                      pin_memory=torch.cuda.is_available())

In [ ]:
x = next(iter(dataloader_train))

In [ ]:
data = x['data'] # [batch_size x ways x shots x image_dim]

In [ ]:
data = data.unsqueeze(0)

In [ ]:
data.shape

torch.Size([1, 5, 4, 1, 28, 28])

In [ ]:
batch_size = data.size(0)
ways = data.size(1)

In [ ]:
batch_size, ways

(1, 5)

In [ ]:
x_support = data[:,:,:1]

In [ ]:
x_support = x_support.reshape((batch_size, ways * 1, *x_support.shape[-3:])) # e.g. [1,50*n_support,*(3,84,84)]
x_query = data[:,:, 1:]
x_query = x_query.reshape((batch_size, ways * 3, *x_query.shape[-3:]))

In [ ]:
x_query.shape

torch.Size([1, 15, 1, 28, 28])

In [ ]:
x_support.shape

torch.Size([1, 5, 1, 28, 28])

In [ ]:
x = torch.cat([x_support, x_query], 1) # e.g. [1,50*(n_support+n_query),*(3,84,84)]

In [ ]:
ae = CNN_4Layer(in_channels=1)

In [ ]:
_, op = ae.forward(x)

In [ ]:
op.shape

torch.Size([20, 1, 1, 1])